In [1]:
import sys
import pandas as pd # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore

sys.path.append("../")
from src import soporte_limpieza as sp_lim

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.float_format', lambda x: '%.2f' % x)

path_ficheros_salida = '../datos/output/'
fichero = 'datos-limpios-2013-2021.csv'

df = pd.read_csv(path_ficheros_salida + fichero, sep = ';', index_col = 0)

convertimos la fecha, ya que viene como string

In [2]:
df = sp_lim.l_limpieza_conversion_fecha(df)

In [ ]:
df.head(1)

In [ ]:
df.dtypes

In [ ]:
df.describe(include = "object").T

In [ ]:
df.describe().T

# objetivo
Desviaciones entre lo previsto y lo recaudado: Determinar en qué categorías económicas o tipos de ingresos las diferencias son más pronunciadas.

Evolución temporal de la recaudación: Identificar cómo han cambiado las previsiones y recaudaciones año a año, y si existen patrones temporales, como meses específicos donde hay mayores discrepancias.

Rendimiento por órgano y unidad gestora: Evaluar qué órganos o unidades gestoras son más eficientes en términos de alcanzar las metas de recaudación y cuáles presentan consistentemente una baja ejecución.

##### Distribución de Ingresos por Categoría Económica:

1.  Analizar las categorías de ingresos más significativas y su participación en los ingresos totales.

In [ ]:
suma_total = df['VALOR_REALIZADO'].sum()
df_categorias_ingresos = df.groupby('CATEGORIA_ECONOMICA')['VALOR_REALIZADO'].sum().reset_index()
df_categorias_ingresos["%"] = np.round(df_categorias_ingresos['VALOR_REALIZADO'] / suma_total * 100, 2)
df_categorias = df_categorias_ingresos.sort_values("%",ascending=False)
display(df_categorias)

2.  Calcular la diferencia promedio entre ingresos previstos y realizados por cada categoría.

In [ ]:
df_categorias_ingresos_promedio = df.groupby(['CATEGORIA_ECONOMICA'])[['VALOR_PREVISTO_ACTUALIZADO', 'VALOR_REALIZADO' ]].mean()
df_categorias_ingresos_promedio['DIFERENCIA_PROMEDIO'] = np.round(df_categorias_ingresos_promedio['VALOR_REALIZADO'] - df_categorias_ingresos_promedio['VALOR_PREVISTO_ACTUALIZADO'])
df_categorias_ingresos_promedio = df_categorias_ingresos_promedio.reset_index()
display(df_categorias_ingresos_promedio)

2.A  Rendimiento organo

In [ ]:
df_temp_2a = df
df_temp_2a['ANIO_LANZAMIENTO'] = pd.to_datetime(df_temp_2a['FECHA_LANZAMIENTO']).dt.year

df_organo_ingresos_promedio = df_temp_2a.groupby(['NOMBRE_DEL_ORGANO'])[['VALOR_PREVISTO_ACTUALIZADO', 'VALOR_REALIZADO' ]].mean()
df_organo_ingresos_promedio['DIFERENCIA_PROMEDIO'] = np.round(df_organo_ingresos_promedio['VALOR_REALIZADO'] - df_organo_ingresos_promedio['VALOR_PREVISTO_ACTUALIZADO'])
df_organo_ingresos_promedio = df_organo_ingresos_promedio.reset_index()
#df_organo_ingresos_promedio['ANIO_LANZAMIENTO'] = df_organo_ingresos_promedio['ANIO_LANZAMIENTO'].astype(int)
df_organo_ingresos_promedio = df_organo_ingresos_promedio.nsmallest(13, 'DIFERENCIA_PROMEDIO')
df_organo_ingresos_promedio.head(1)

##### Análisis Temporal:

1.  Evaluar las tendencias a lo largo del tiempo, por ejemplo, cómo cambian los ingresos realizados de un mes a otro o de un año a otro.
    

In [ ]:
df_temp = df
df_temp['ANIO_LANZAMIENTO'] = pd.to_datetime(df_temp['FECHA_LANZAMIENTO']).dt.year
df_ingresos_tiempo = df_temp.groupby(['ANIO_LANZAMIENTO','CATEGORIA_ECONOMICA'])[['VALOR_REALIZADO']].sum().reset_index()
df_ingresos_tiempo['ANIO_LANZAMIENTO'] = df_ingresos_tiempo['ANIO_LANZAMIENTO'].astype(int)
df_ingresos_tiempo.head(1)

In [ ]:
df_temp_pre = df
df_temp_pre['ANIO_LANZAMIENTO'] = pd.to_datetime(df_temp_pre['FECHA_LANZAMIENTO']).dt.year
df_ingresos_tiempo_pre = df_temp_pre.groupby(['ANIO_LANZAMIENTO','CATEGORIA_ECONOMICA'])[['VALOR_PREVISTO_ACTUALIZADO']].sum().reset_index()
df_ingresos_tiempo_pre['ANIO_LANZAMIENTO'] = df_ingresos_tiempo_pre['ANIO_LANZAMIENTO'].astype(int)
df_ingresos_tiempo_pre.head(1)

2.  Identificación de Discrepancias:
    Investigar las categorías con mayor diferencia entre lo previsto y lo realizado, identificando patrones en la subejecución o sobre ejecución.

In [ ]:
df_temp_dis = df
df_temp_dis['ANIO_LANZAMIENTO'] = pd.to_datetime(df_temp_dis['FECHA_LANZAMIENTO']).dt.year
df_categorias_ingresos_promedio_anios = df_temp_dis.groupby(['ANIO_LANZAMIENTO','CATEGORIA_ECONOMICA'])[['VALOR_PREVISTO_ACTUALIZADO', 'VALOR_REALIZADO' ]].mean()
df_categorias_ingresos_promedio_anios['DIFERENCIA_PROMEDIO'] = np.round(df_categorias_ingresos_promedio_anios['VALOR_REALIZADO'] - df_categorias_ingresos_promedio_anios['VALOR_PREVISTO_ACTUALIZADO'])
df_categorias_ingresos_promedio_anios = df_categorias_ingresos_promedio_anios.reset_index()
df_categorias_ingresos_promedio_anios['ANIO_LANZAMIENTO'] = df_categorias_ingresos_promedio_anios['ANIO_LANZAMIENTO'].astype(int)
df_categorias_ingresos_promedio_anios = df_categorias_ingresos_promedio_anios.reset_index()

df_categorias_ingresos_promedio_anios.sample(1)


## Fase 4: Visualización de Datos

1. **Gráficos de Barras y Líneas:**

   - Crear gráficos que muestren la comparación entre ingresos previstos, lanzados y realizados para cada categoría.

   - Graficar la evolución temporal de los ingresos realizados y previstos.

In [ ]:
plt.figure(figsize = (7,5))

sns.barplot(y="CATEGORIA_ECONOMICA", 
            hue="%",
            x="%",
            data=df_categorias,
            palette="tab10",
        )

plt.title("Porcentaje de ingreso por Categoria")
plt.xlabel("")
plt.ylabel("")
plt.xticks(rotation=0)
plt.yticks()
plt.show()

In [ ]:
plt.figure(figsize = (8,5))

sns.barplot(y="CATEGORIA_ECONOMICA", 
            hue="DIFERENCIA_PROMEDIO",
            x="DIFERENCIA_PROMEDIO",
            data=df_categorias_ingresos_promedio,
            palette="tab10",
        )

plt.title("Diferencia promedio entre ingresos previstos y realizados por categorias")
plt.xlabel("variacion")
plt.ylabel("")
plt.xticks(rotation=0)
plt.yticks()
plt.show()

In [ ]:
plt.figure(figsize = (13,6))

sns.pointplot(
            x="ANIO_LANZAMIENTO",
            y="VALOR_REALIZADO",
            data=df_ingresos_tiempo,
            palette="tab10",
            hue="CATEGORIA_ECONOMICA",
        )

plt.title("Evolucion de ingresos Realizados por categorias")
plt.xlabel("")
plt.ylabel("Ingresos(M)")
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize = (13,6))

sns.pointplot(
            x="ANIO_LANZAMIENTO",
            y="VALOR_PREVISTO_ACTUALIZADO",
            data=df_ingresos_tiempo_pre,
            palette="tab10",
            hue="CATEGORIA_ECONOMICA",
        )

plt.title("Evolucion de ingresos Previstos por categorias")
plt.xlabel("")
plt.ylabel("Ingresos(M)")
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize = (13,10))

sns.barplot(
            x="DIFERENCIA_PROMEDIO",
            y="NOMBRE_DEL_ORGANO", 
            data=df_organo_ingresos_promedio,
            palette="tab10",
            hue="NOMBRE_DEL_ORGANO",
        )

plt.title("Diferencia promedio de previsto, ingresado por Organo")
plt.xlabel("")
plt.ylabel("")
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize = (11,6))

sns.scatterplot(
            x="ANIO_LANZAMIENTO",
            y="DIFERENCIA_PROMEDIO",
            data=df_categorias_ingresos_promedio_anios,
            palette="tab10",
            hue="CATEGORIA_ECONOMICA",
        )

plt.title("Evolucion de diferencia de promedios por categorias y años")
plt.xlabel("")
plt.ylabel("Promedio")
plt.xticks(rotation=0)
plt.show()